\_\_init_subclass instead of metaclass
Can use as class register

meta class bad because makes multiple inheritance really hard

Show how things in python work like this:

In [ ]:
class FixedCustomer:
    first_name = Field()
    last_name = Field()
    

prefer class decorators over metaclasses for composing class extensions